In [1]:
using Gmsh
using Gridap
using GridapGmsh
using SparseArrays
using Gridap.Geometry
using ChainRules
using Zygote
using NLopt
import Gmsh: gmsh
import ChainRules: rrule

Gridap.outer(a::Number,b::Number) = a * b
Gridap.inner(a::Number,b::Number) = a * b
Gridap.Helpers.operate(::typeof(tanh),x::Float64)=tanh(x)

include("MeshGenerator.jl")
include("PML.jl")
include("Helper.jl")
include("FilterAndThreshold.jl")
include("Objective.jl")
# Geometry parameters of the mesh
L = 4.0           # Length of the normal region
h1 = 3.0          # Height of the upper region
h2 = 1.5          # Height of the lower region above source
h3 = 1.5          # Height of the lower region below source
dpml = 1.0        # Thickness of the PML
hd = 1.0          # Height of the design region
LH = [L,h1,h2+h3]
# Characteristic length (controls the resolution, smaller the finer)
λ = 1.0           # Wavelength (aribitrary unit)
resol = 20.0      # Number of points per wavelength
l0 = λ/resol      # Normal region
ld = l0           # Design region
lpml = 2*l0       # PML 

# Target region size
Lt = 1.0          # Length
ht = 1.0          # Height

# Physical parameters 
k = 2*π/λ        # Wave number 
ω = k            # c=1
ϵ1 = 1.0        # Relative electric permittivity for material 1
ϵ2 = 12.0       # Relative electric permittivity for material 2
μ = 1.0          # Relative magnetic permeability for all materials

# PML parameters
R = 1e-4         # Tolerence for PML reflection 
σ = -3/4*log(R)/dpml/sqrt(ϵ1)

# Source term (point source approximation)
pos = -h2-hd  # Source location
δ = l0/5.0    # Gaussian source width


MeshGenerator(L,h1,h2,h3,hd,Lt,ht,dpml,l0,ld,lpml)
include("GridapSetup.jl")

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 2 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Meshing curve 5 (Line)
Info    : Meshing curve 6 (Line)
Info    : Meshing curve 7 (Line)
Info    : Meshing curve 8 (Line)
Info    : Meshing curve 9 (Line)
Info    : Meshing curve 10 (Line)
Info    : Meshing curve 11 (Line)
Info    : Meshing curve 12 (Line)
Info    : Meshing curve 13 (Line)
Info    : Meshing curve 14 (Line)
Info    : Meshing curve 15 (Line)
Info    : Meshing curve 16 (Line)
Info    : Meshing curve 17 (Line)
Info    : Meshing curve 18 (Line)
Info    : Meshing curve 19 (Line)
Info    : Meshing curve 20 (Line)
Info    : Meshing curve 21 (Line)
Info    : Meshing curve 22 (Line)
Info    : Meshing curve 23 (Line)
Info    : Meshing curve 24 (Line)
Info    : Meshing curve 25 (Line)
Info    : Meshing curve 26 (Line)
Info    : Mes

GridapFEM (generic function with 1 method)

In [7]:
# Filter and threshold paramters
r = ld*2.0       # Filter radius
β = 15.0        # β∈[1,∞], threshold sharpness
η = 0.5          # η∈[0,1], threshold center

# Loss control
α = 0.

It = 1.0
I = 2*π*It           # Source amplitude

linesource = true
flag_t = true    # Turn on/off the threshold
flag_f = true    # Turn on/off the filter
 
opt = Opt(:LD_MMA, np)
opt.lower_bounds = 0.0
opt.upper_bounds = 1.0
opt.ftol_rel = 1e-3
#opt.stopval = 0.0505*L*h1
opt.maxeval = 200
opt.min_objective = g_p

#(g_opt,p_opt,ret) = optimize(opt, ones(np)*0.5)
(g_opt,p_opt,ret) = optimize(opt, p)
@show numevals = opt.numevals # the number of function evaluations

# Display u and ε
p = p_opt
#p = rand(np)
pf = pf_p(p)
uvec = u_pf(pf)./It
ϵ0 = ϵ1 + (ϵ2-ϵ1)*FEFunction(P,p_vec(p,P,tags,design_tag))
if (flag_f)
    ϵt = ϵ1 + (ϵ2-ϵ1)*Threshold(β,η,FEFunction(Pf,pf),flag_t)
else
    ϵt = ϵ1 + (ϵ2-ϵ1)*Threshold(β,η,FEFunction(P,pf),flag_t)
end
f2_target(x) = ((abs(x[1])<=Lt/2)&&(abs(x[2]-h1/2)<=ht/2))
diff = myabs2(myabs2(FEFunction(U,uvec))-f2_target)
writevtk(trian,"demo",cellfields=["diff"=>diff,"ϵ0"=>ϵ0,"ϵt"=>ϵt,"Real(E)"=>FEFunction(U,real(uvec)),"Imag(E)"=>FEFunction(U,imag(uvec)),"|E|^2"=>FEFunction(U,(real(uvec).^2+imag(uvec).^2))])
diff1 = restrict(diff,trian_t1)
diff2 = restrict(diff,trian_t2)
(sum(integrate(diff1,trian_t1,quad_t1))+sum(integrate(diff2,trian_t2,quad_t2)))/(L*h1)

g_value = g_p(p) = 0.9666179086552592
g_value = g_p(p) = 0.9327333557399917
g_value = g_p(p) = 0.7805538135506813
g_value = g_p(p) = 1.0037833161700596
g_value = g_p(p) = 0.8981039885765705
g_value = g_p(p) = 0.7459014386231323
g_value = g_p(p) = 0.6828293185129024
g_value = g_p(p) = 0.6870573663543711
g_value = g_p(p) = 0.6496338119910225
g_value = g_p(p) = 0.6982175685621688
g_value = g_p(p) = 0.6418054778823883
g_value = g_p(p) = 0.6370891663810299
g_value = g_p(p) = 0.6585962391060863
g_value = g_p(p) = 0.6363213478456035
g_value = g_p(p) = 0.6359566922140988
g_value = g_p(p) = 0.633667216869198
g_value = g_p(p) = 0.6296580907123278
g_value = g_p(p) = 1.1333237533403906
g_value = g_p(p) = 0.6417907611269256
g_value = g_p(p) = 0.6291722641538271
g_value = g_p(p) = 0.6286661474646924
g_value = g_p(p) = 0.6277456618776043
g_value = g_p(p) = 0.6274562595811539
g_value = g_p(p) = 0.6243178897462931
g_value = g_p(p) = 0.7420570670937419
g_value = g_p(p) = 0.6397254873876895
g_value = g_p

0.05191651274336298

In [56]:
g_p(p_opt)

0.6199431608764043

In [30]:
using Richardson
using Random
p0 = rand(np)
#p0=p_opt
#Random.seed!(1)
δp = rand(np)
#δp = rand(num_free_dofs(P))
#flag_t = true
#flag_f = true
#include("GridapSetup.jl")
#include("FilterAndThreshold.jl")
#include("Objective.jl")
extrapolate(δp*0.1, rtol=0) do h
    @show norm(h)
    (g_p(h+p0)-g_p(p0)) / norm(h)
end

norm(h) = 4.309036726698927
norm(h) = 0.5386295908373658
norm(h) = 0.06732869885467073
norm(h) = 0.008416087356833841
norm(h) = 0.0010520109196042301
norm(h) = 0.00013150136495052877
norm(h) = 1.6437670618816096e-5


(-0.0308899507138315, 3.9116175576392465e-12)

In [31]:
dgdp=zeros(np)
#dgdp=zeros(num_free_dofs(P))
g_p(p0,dgdp)
dgdp'*δp/norm(δp)

g_value = g_p(p) = 1.444256160044532


-0.030889950641654843

In [55]:
include("Objective.jl")

g_p (generic function with 2 methods)

In [52]:
g_u(uvec)

(0.6199431608764043, 2.242503010452102)